In [5]:
! ls data/

atc_codes.csv                       methods.csv
atc_codes_clean.csv                 methods_2.csv
companies_drugs_keyed.csv           pharma_lobby.csv
data.csv                            spending_2011.csv
data_2.csv                          spending_2012.csv
drug_list.csv                       spending_2013.csv
drug_uses.csv                       spending_2014.csv
drugdata_clean.csv                  spending_2015.csv
drugnames_withclasses.csv           spending_all_top100.csv
fda_ndc_product.csv                 spending_part_b_2011to2015_tidy.csv
lobbying_keyed.csv                  usp_drug_classification.csv
manufacturers_drugs_cleaned.csv     variables.csv
meps_full_2014.csv                  variables_2.csv


In [1]:
import pandas as pd

### Concatenating all spending datasets into one data frame with 'year' and 'key'.

In [71]:
sp_2011 = pd.read_csv('data/spending_2011.csv')
sp_2012 = pd.read_csv('data/spending_2012.csv')
sp_2013 = pd.read_csv('data/spending_2013.csv')
sp_2014 = pd.read_csv('data/spending_2014.csv')
sp_2015 = pd.read_csv('data/spending_2015.csv')

## Year for further filtering and data slicing. 

sp_2011['year']=2011
sp_2012['year']=2012
sp_2013['year']=2013
sp_2014['year']=2014
sp_2015['year']=2015

## dummy primary key. I'm adding since we are merging all spending files into one. This will not affect the data.

sp_2011['key']= 52011+sp_2011['column_a']
sp_2012['key']= 62012+sp_2012['column_a']
sp_2013['key']= 72013+sp_2013['column_a']
sp_2014['key']= 82014+sp_2014['column_a']
sp_2015['key']= 92015+sp_2015['column_a']

## Merging all datframes into one.

merge_spend_df = pd.concat([sp_2011,sp_2012, sp_2013, sp_2014, sp_2015], ignore_index=True)

In [72]:
merge_spend_df.shape

## Notice the increase in fields. We added 'Year' and 'Key'.

(17302, 15)

In [75]:
merge_spend_df.groupby(['year'])['column_a'].count()

## Group by on merged data frame gives year wise breakdown. Sum of these equals 17302.

year
2011    3583
2012    3510
2013    3460
2014    3359
2015    3390
Name: column_a, dtype: int64

In [76]:
sp_drug_list = merge_spend_df['drugname_generic'].unique().tolist()
len(sp_drug_list)

## 2178 unique drug generic names in merged spending dataframe. 

2178

### Creating data frame from drug uses dataset

In [94]:
drug_use = pd.read_csv('data/drug_uses.csv')
drug_use.shape

(6183, 9)

In [95]:
ds_drug_list = drug_use['drugname_generic'].unique().tolist()
len(ds_drug_list)

## 574 unique drug generic names

574

### Now, let's join the merged spending data with drug data on "drugname_generic" field. This will combine fields from both datasets

In [106]:
merge_df = merge_spend_df.merge(drug_use, on='drugname_generic', how='inner')
merge_df.shape

## NOTICE - Increase in fields. Fields from drug data along with spending data.

(212977, 23)

In [107]:
len(merge_df['drugname_generic'].unique().tolist())

## 574 Matching drug names. Out of 2178 distinct generic drug names from merged spending dataframe.
## Remember, 574 matches are distributed across various years from 2011 to 2015.

574

## Using "set", quickly validating the above step. 

In [97]:
sp_drug_list = merge_spend_df['drugname_generic'].unique().tolist() #generic drug name from merged spending dataframe. 
ds_drug_list = drug_use['drugname_generic'].unique().tolist() ## generic drug name from drug_uses.csv

In [98]:
## Validating the drug match between merged spending dataframe and drug_use dataframe

a = set(ds_drug_list) 
b = set(sp_drug_list) 

In [99]:
result_set = list(a.intersection(b))
len(result_set)

## number matches with distinct drug generic names from merged data frame above. 

574

### Validation and summary numbers for quick understanding

In [100]:
# We have 3 things 
## a) merge_spend_df - merged dataframe containing spending data from 2011 to 2015
## b) drug_use - drug dataframe containing generic drug names
## c) merge_df - Merged(inner joined) dataframe created from a & b on key "drugname_generic". 

metric_df = pd.DataFrame()
metric_df['distinct_total'] = merge_spend_df.groupby(['year'])['column_a'].count() # distinct rows from spending data
metric_df['distinct_match']= merge_df.groupby(['year'])['key'].nunique() # distinct match(drug data) from spending data
metric_df['no_match'] = metric_df['distinct_total'] - metric_df['distinct_match'] #calculated difference. 
metric_df.head()

,distinct_total,distinct_match,no_match
year,,,
2011,3583,945,2638
2012,3510,951,2559
2013,3460,972,2488
2014,3359,992,2367
2015,3390,1009,2381


In [103]:
## Let's filter out unmatched spending data and calculate numbers. 
## Expectation -> it should match with no_match from above

# Creating filter to extract 'key' as list from merged spending data.

drug_column_id = merge_df['key'].unique().tolist()
drug_unmatched_filter = merge_spend_df['key'].isin(drug_column_id)

# applying INVERSE filter to get un matched spending data.

drug_unmatched_df = merge_spend_df[~drug_unmatched_filter]
drug_unmatched_df.groupby(['year'])['key'].nunique()

## NOTE - the numbers are matching with "no_match" calculated data. 

year
2011    2638
2012    2559
2013    2488
2014    2367
2015    2381
Name: key, dtype: int64

In [104]:
drug_unmatched_df.shape

(12433, 15)

### Writing data into CSV.. 

In [105]:
merge_spend_df.to_csv('data/spending_data_merged.csv') # merged spending data from 2011 to 2015
merge_df.to_csv('data/spending_data_match.csv') # matched (drug data - generic drug name) spending data
drug_unmatched_df.to_csv('data/spending_data_unmatched.csv') #unmatched spending data